# Solving Sudoku

In [8]:
from sudoku import solve_sudoku, draw_sudoku

def test_sudoku(constraints):
    status, solution = solve_sudoku(constraints)
    return "PASS" if status == 'Optimal' else "FAIL"

## Constraints
Tuples of (x, y, number)

In [11]:
constraints = [(1,1,9), (1,2,3), (8,3,7), (2,2,9)]
print(draw_sudoku(constraints))

-------------------------------
| 9  -  - | -  -  - | -  -  - |
| 3  9  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  7  - |
-------------------------------
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------



## The Delta Debugging Algorithm

In [43]:
from ddebug import get_partitions
from covenant import pre, post

@pre(lambda data, test, granularity: test(data)=='FAIL')
@post(lambda result, data, test, granularity: test(result)=='FAIL')

def delta_debug(data, test, granularity=2):
    print('\n"{2}", granularity={1}'.format(len(data), granularity, data))    
    for subset in get_partitions(data, granularity):
        result = test(subset)
        print('"{}" -> {}'.format(subset, result))
        
        if result == 'FAIL':
            if len(subset) > 1:
                return delta_debug(subset, test, granularity)
            else:
                return subset # minimal failing subset
            
    if granularity < len(data):
        return delta_debug(data, test, granularity + 1)
    return data

## Simple string example

In [44]:
def no_rats(s): return "FAIL" if 'rat' in s else "PASS"

text = "ten wooden crates"
delta_debug(text, no_rats)


"ten wooden crates", granularity=2
"en crates" -> FAIL

"en crates", granularity=2
"rates" -> FAIL

"rates", granularity=2
"tes" -> PASS
"ra" -> PASS

"rates", granularity=3
"ates" -> PASS
"res" -> PASS
"rat" -> FAIL

"rat", granularity=3
"at" -> PASS
"rt" -> PASS
"ra" -> PASS


'rat'

## Solving a Sudoku

In [46]:
constraints = [(1,1,9), (1,2,3), (8,3,7), (2,2,9)]
minimal = delta_debug(constraints, test_sudoku)
print(draw_sudoku(minimal))


"[(1, 1, 9), (1, 2, 3), (8, 3, 7), (2, 2, 9)]", granularity=2
"[(8, 3, 7), (2, 2, 9)]" -> PASS
"[(1, 1, 9), (1, 2, 3)]" -> PASS

"[(1, 1, 9), (1, 2, 3), (8, 3, 7), (2, 2, 9)]", granularity=3
"[(1, 2, 3), (8, 3, 7), (2, 2, 9)]" -> PASS
"[(1, 1, 9), (8, 3, 7), (2, 2, 9)]" -> FAIL

"[(1, 1, 9), (8, 3, 7), (2, 2, 9)]", granularity=3
"[(8, 3, 7), (2, 2, 9)]" -> PASS
"[(1, 1, 9), (2, 2, 9)]" -> FAIL

"[(1, 1, 9), (2, 2, 9)]", granularity=3
"[(2, 2, 9)]" -> PASS
"[(1, 1, 9)]" -> PASS
-------------------------------
| 9  -  - | -  -  - | -  -  - |
| -  9  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------



## More complicated Sudoku example

In [32]:
constraints = [(1,1,1), (2,1,2), (3,1,3), (4,2,1), (5,2,2), (6,2,3), (7,3,7), (8,3,8), (9,3,9)]
print(draw_sudoku(constraints))
minimal = delta_debug(constraints, test_sudoku)
print(draw_sudoku(minimal))

-------------------------------
| 1  2  3 | -  -  - | -  -  - |
| -  -  - | 1  2  3 | -  -  - |
| -  -  - | -  -  - | 7  8  9 |
-------------------------------
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
| -  -  - | -  -  - | -  -  - |
-------------------------------

"[(5, 2, 2), (6, 2, 3), (7, 3, 7), (8, 3, 8), (9, 3, 9)]" -> PASS
"[(1, 1, 1), (2, 1, 2), (3, 1, 3), (4, 2, 1)]" -> PASS
"[(4, 2, 1), (5, 2, 2), (6, 2, 3), (7, 3, 7), (8, 3, 8), (9, 3, 9)]" -> PASS
"[(1, 1, 1), (2, 1, 2), (3, 1, 3), (7, 3, 7), (8, 3, 8), (9, 3, 9)]" -> PASS
"[(1, 1, 1), (2, 1, 2), (3, 1, 3), (4, 2, 1), (5, 2, 2), (6, 2, 3)]" -> PASS
"[(3, 1, 3), (4, 2, 1), (5, 2, 2), (6, 2, 3), (7, 3, 7), (8, 3, 8), (9, 3, 9)]" -> FAIL
"[(4, 2, 1), (5, 2, 2), (6, 2, 3), (7, 3, 7), (8, 3, 8), (9, 3, 9)]" -> PASS
"[(3, 1, 3), (6, 2, 3), (7, 3, 7), (8, 3, 8), (9, 3, 9)]" -> FAIL
